In [14]:
%load_ext autoreload
%autoreload 2
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine as cosine_distance
from scipy.special import comb as num_combinations
import numpy as np
from itertools import combinations
from tqdm import tqdm
import sys
sys.path.append('../src')
from models import produce_effect_size, calculate_association_metric_for_target_word, get_matrices_from_term_lists
from models import produce_p_value

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
wv_from_bin = KeyedVectors.load_word2vec_format("../data/external/googlenews/GoogleNews-vectors-negative300.bin", binary=True)  # C bin format

In [20]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']

In [16]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']

In [17]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']

In [5]:
wv_test_object = {'x_word1':[.1,.2,.3],
            'x_word2':[.4,.5,.6],
            'y_word1':[.2,.4,.6],
            'y_word2':[.1,.3,.5],
            'a_word1':[.5,.7,.9],
            'a_word2':[.4,.6,.8],
            'b_word1':[.3,.6,.9],
            'b_word2':[.4,.7,.9]}

In [5]:
X_terms = ['x_word1', 'x_word2']
Y_terms = ['y_word1', 'y_word2']
A_terms = ['a_word1', 'a_word2']
B_terms = ['b_word1', 'b_word2']
[X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

In [61]:
produce_effect_size(wv_test_object, X_terms, Y_terms, A_terms, B_terms)

1.1899801283181513

# Test Statistic

In [66]:
x1_vec = wv_test_object['x_word1']
s1 = calculate_association_metric_for_target_word(x1_vec, A_mtx, B_mtx)

In [68]:
x2_vec = wv_test_object['x_word2']
s2 = calculate_association_metric_for_target_word(x2_vec, A_mtx, B_mtx)

In [74]:
y1_vec = wv_test_object['y_word1']
sy1 = calculate_association_metric_for_target_word(y1_vec, A_mtx, B_mtx)
y2_vec = wv_test_object['y_word2']
sy2 = calculate_association_metric_for_target_word(y2_vec, A_mtx, B_mtx)

In [75]:
print(s1, s2, s1+s2 - (sy1+sy2))

-0.007639423266564194 0.013706608780193585 0.03175706038191817


In [8]:
def get_complements(x_union_y):
    '''Generator function that yields pairs of equal-size disjoint subsets
    of x_union_y.
    x_union_y should a set type.'''
    already_seen = set()
    for seq in combinations(x_union_y, len(x_union_y)//2):
        complement = frozenset(x_union_y.difference(seq))
        #already_seen.append(complement)
        already_seen.add(complement)
        #if frozenset(seq) in already_seen:
            #continue
        yield (seq, complement)

In [5]:
x_union_y = {'a','b','c','d'}
for tup in get_complements(x_union_y):
    print(tup)

(('c', 'd'), frozenset({'b', 'a'}))
(('c', 'b'), frozenset({'d', 'a'}))
(('c', 'a'), frozenset({'d', 'b'}))


In [9]:
def produce_p_value(wv_obj, X_terms, Y_terms, A_terms, B_terms):
    x_union_y = set(X_terms).union(set(Y_terms))
    total_terms = len(x_union_y)
    total_pairs = 0
    high_test_statistics = 0
    comparison_statistic = produce_test_statistic(wv_obj, X_terms, Y_terms, A_terms, B_terms)
    for (X_i_terms, Y_i_terms) in tqdm(get_complements(x_union_y), total=num_combinations(total_terms, total_terms/2)):
        total_pairs += 1
        test_statistic = produce_test_statistic(wv_obj, X_i_terms, Y_i_terms, A_terms, B_terms)
        if (test_statistic > comparison_statistic): high_test_statistics += 1
    return (float(high_test_statistics) / float(total_pairs))
produce_p_value(wv_from_bin, X_terms, Y_terms, A_terms, B_terms)

100%|██████████████████████████████████████████████████████████████████████████| 12870/12870.0 [03:16<00:00, 65.34it/s]


0.003962703962703963

In [21]:
produce_p_value(wv_from_bin, X_terms, Y_terms, A_terms, B_terms)


  0%|                                                                                      | 0/12870.0 [00:00<?, ?it/s]
  0%|                                                                              | 9/12870.0 [00:00<02:36, 82.31it/s]
  0%|                                                                             | 17/12870.0 [00:00<02:42, 78.98it/s]
  0%|▏                                                                            | 26/12870.0 [00:00<02:40, 79.96it/s]
  0%|▏                                                                            | 34/12870.0 [00:00<02:45, 77.78it/s]
  0%|▎                                                                            | 44/12870.0 [00:00<02:37, 81.44it/s]
  0%|▎                                                                            | 52/12870.0 [00:00<02:42, 78.77it/s]
  0%|▎                                                                            | 61/12870.0 [00:00<02:40, 79.80it/s]
  1%|▍                                 

7.854364027881644e-05

# Results

Weat 6: 0 (0/6435)
Weat 7: 0.02905 (187/6435)
Weat 8: 0.0003108 (2/6435)

Results without complements: Weat 6: 0 (10e-4), Weat 7: 0.0225 (.027); Weat 8: 0.00396 (10e-2)

Results after modification: WEAT 6:  7.8543e-5 WEAT 7: 0.0266.  WEAT 8: .00643